In [2]:
import numpy as np
import pandas as pd

DATA_DIR = "data"

In [3]:
catboost_version = 1
lightgbm_version = 1
num_fold = 5

y = pd.read_csv(DATA_DIR + '/submission_format.csv', index_col='building_id')

y_catboost_test = np.load(DATA_DIR + '/intermediate/catboost_{}fold_v{}_test.npy'.format(num_fold, catboost_version))
y_lightgbm_test = np.load(DATA_DIR + '/intermediate/lightgbm_{}fold_v{}_test.npy'.format(num_fold, lightgbm_version))

y_catboost_df = pd.DataFrame(data = y_catboost_test, columns = ["1","2","3"], index=y.index)
y_lightgbm_df = pd.DataFrame(data = y_lightgbm_test, columns = ["1","2","3"], index=y.index)

y_lightgbm_df

,1,2,3
building_id,,,
300051,0.000994,0.278134,0.720872
99355,0.001265,0.916416,0.082320
890251,0.012274,0.790961,0.196765
745817,0.678771,0.319503,0.001727
421793,0.000344,0.186396,0.813260
...,...,...,...
310028,0.066407,0.874372,0.059221
663567,0.001021,0.319639,0.679340
1049160,0.033148,0.833649,0.133203


In [4]:
y_catboost_1_2_test = np.load(DATA_DIR + '/intermediate/catboost_{}fold_1_2_test.npy'.format(num_fold))
y_lightgbm_1_2_test = np.load(DATA_DIR + '/intermediate/lightgbm_{}fold_1_2_test.npy'.format(num_fold))

y_catboost_2_3_test = np.load(DATA_DIR + '/intermediate/catboost_{}fold_2_3_test.npy'.format(num_fold))
catboost_gscv_2_3_test = np.load(DATA_DIR + '/intermediate/catboost_gscv_test.npy')
y_lightgbm_2_3_test = np.load(DATA_DIR + '/intermediate/lightgbm_{}fold_2_3_test.npy'.format(num_fold))

y_catboost_1_2_df = pd.DataFrame(data=y_catboost_1_2_test, columns=["1","2"], index=y.index)
y_lightgbm_1_2_df = pd.DataFrame(data=y_lightgbm_1_2_test, columns=["2"], index=y.index)
y_lightgbm_1_2_df["1"] = 1 - y_lightgbm_1_2_df["2"]

y_catboost_2_3_df = pd.DataFrame(data=y_catboost_2_3_test, columns=["2","3"], index=y.index)
catboost_gscv_2_3_df = pd.DataFrame(data=catboost_gscv_2_3_test,columns=["2","3"],index=y.index)
y_lightgbm_2_3_df = pd.DataFrame(data=y_lightgbm_2_3_test, columns=["3"], index=y.index)
y_lightgbm_2_3_df["2"] = 1 - y_lightgbm_2_3_df["3"]

y_catboost_2_3_df

,2,3
building_id,,
300051,0.282369,0.717631
99355,0.925907,0.074093
890251,0.733721,0.266279
745817,0.997244,0.002756
421793,0.237181,0.762819
...,...,...
310028,0.937492,0.062508
663567,0.360147,0.639853
1049160,0.844550,0.155450


In [5]:
weight = [0.5, 0.5]
y_weighted_test = y_catboost_df.multiply(weight[0]).add(y_lightgbm_df.multiply(weight[1]))

y_weighted_1_2_test = y_catboost_1_2_df.add(y_lightgbm_1_2_df)
# y_weighted_1_2_test = y_catboost_1_2_df.add(y_lightgbm_1_2_df).add(y_weighted_test.drop(columns="3",axis=1))

y_7534 = pd.read_csv(DATA_DIR + '/submission_7534.csv', index_col='building_id')

y_weighted_2_3_test = y_catboost_2_3_df.add(y_lightgbm_2_3_df)
y_gscv_2_3_test = catboost_gscv_2_3_df.add(y_lightgbm_2_3_df)
# y_weighted_2_3_test = y_catboost_2_3_df.add(y_lightgbm_2_3_df).add(y_weighted_test.drop(columns="1",axis=1))

y_weighted_test

,1,2,3
building_id,,,
300051,0.000834,0.274864,0.724302
99355,0.001664,0.915487,0.082848
890251,0.014886,0.762626,0.222488
745817,0.709049,0.289734,0.001217
421793,0.000328,0.196497,0.803175
...,...,...,...
310028,0.077058,0.862169,0.060774
663567,0.001091,0.378627,0.620283
1049160,0.040354,0.801287,0.158359


In [6]:
y_submission = y

conflicts = 0
changes = {"1":{},"2":{},"3":{}}

diff_gscv = pd.DataFrame(columns=y_gscv_2_3_test.columns)
diff_5fold = pd.DataFrame(columns=y_weighted_2_3_test.columns)

for index, row in y_weighted_test.iterrows():
    pred_3 = row.idxmax()
    # pred_3 = str(row["damage_grade"])
    pred_1_2 = y_weighted_1_2_test.loc[index].idxmax()
    pred_2_3 = y_gscv_2_3_test.loc[index].idxmax()
    # pred_2_3_5fold = y_weighted_2_3_test.loc[index].idxmax()
    # if(pred_2_3!=pred_2_3_5fold):
    #     diff_gscv = diff_gscv.append(y_gscv_2_3_test.loc[index])
    #     diff_5fold = diff_5fold.append(y_weighted_2_3_test.loc[index])
    pred = pred_3
    # if(pred_3 == "1"):
    #     pred = pred_1_2
    # elif(pred_3 == "3"):
    #     pred = pred_2_3
    # else:
    #     if(pred_1_2 == "1"):
    #         if(pred_2_3 == "3"):
    #             # pred = "2"
    #             conflicts += 1
    #         else:
    #             pred = "1"
    #     else:
    #         pred = pred_2_3
    # if(pred != pred_3):
    #     if(pred in changes[pred_3]):
    #         changes[pred_3][pred] += 1
    #     else:
    #         changes[pred_3][pred] = 1
    # y_weighted_test.loc[index,"pred_1_2"]=pred_1_2
    # y_weighted_test.loc[index,"pred_2_3"]=pred_2_3
    y_weighted_test.loc[index,"pred"]=pred

print(conflicts)
print(changes)

0
{'1': {}, '2': {}, '3': {}}


In [6]:
diff_5fold = diff_5fold.rename(columns={"2":"5fold_2","3":"5fold_3"})
diff_gscv = diff_gscv.rename(columns={"2":"gscv_2","3":"gscv_3"})
diff = pd.concat([diff_5fold,diff_gscv],axis=1)
diff = diff.join(y_lightgbm_2_3_df,how="left")
diff["5fold_2"]=diff["5fold_2"]-diff["2"]
diff["5fold_3"]=diff["5fold_3"]-diff["3"]
diff["gscv_2"]=diff["gscv_2"]-diff["2"]
diff["gscv_3"]=diff["gscv_3"]-diff["3"]
diff

,5fold_2,5fold_3,gscv_2,gscv_3,3,2


mix inf
0
{'1': {'2': 356}, '2': {'1': 253}, '3': {}}

mix inf
2
{'1': {'2': 241}, '2': {'3': 660, '1': 166}, '3': {'2': 564}}

1
{'1': {'2': 356}, '2': {'3': 517, '1': 252}, '3': {'2': 559}}

In [7]:
y_submission["damage_grade"] = y_weighted_test["pred"]
y_submission.to_csv(DATA_DIR + '/submission_7533.csv')